In [ ]:
import pandas as pd
import numpy as np
import json
import os, pickle
import torch
from tqdm import tqdm
import logging
logging.basicConfig(level=logging.ERROR)

import warnings
warnings.simplefilter("ignore", UserWarning)

# Preprocessing

In [ ]:
#para
top_k_smilarity = 5
loss_set = 'bce'    # 'bce', 'focal'

change here the path

In [ ]:
import pickle

# OCR doc_info
with open('/kaggle/input/doc-info/train_doc_info.pkl','rb') as f:
    train_doc_info = pickle.load(f)
with open('/kaggle/input/doc-info/val_doc_info.pkl','rb') as f:
    val_doc_info = pickle.load(f)

#embeddings for doc object
with open('/kaggle/input/e5v-embeddings/train_E5Vembeddings.pkl','rb') as f:
    train_doc_emb = pickle.load(f)
with open('/kaggle/input/e5v-embeddings/val_E5Vembeddings.pkl','rb') as f:
    val_doc_emb = pickle.load(f)

#embeddings for question (no ans)
with open('/kaggle/input/e5v-qaemb/train_e5-v_QAemb.pkl','rb') as f:
    train_question_emb= pickle.load(f)
    # train_df = train_df[['question', 'global_id','pmcid']]
    # train_df['global_id'] = train_df['global_id'].apply(ast.literal_eval)
with open('/kaggle/input/e5v-qaemb/val_e5-v_QAemb.pkl','rb') as f:
    val_question_emb = pickle.load(f)   

In [ ]:
with open('/kaggle/input/doc-info/test_doc_info.pkl','rb') as f:
    test_doc_info = pickle.load(f)

with open('/kaggle/input/e5v-embeddings/test_E5Vembeddings.pkl','rb') as f:
    test_doc_emb = pickle.load(f)

with open('/kaggle/input/e5v-qaemb/test_e5-v_QAemb.pkl','rb') as f:
    test_question_emb = pickle.load(f)   

**OCR doc info from local(page) to global(doc)** 

parents & children

Combine information

In [ ]:
def combine_infomation(doc_info):
    global_id_list = []
    relation_list = []
    position_list = []
    # local(page level) relation to global(doc level) relation
    for name in tqdm(doc_info):
        global_order_id_list = []
        xml_json = doc_info[name]
        parent_child_dict = {}
        parent_box = None
        iterate_id = 0
        id = 0

        #Iterate the pages in each doc
        for p in xml_json['pages']:
            page = xml_json['pages'][p]
            order_obj = page['ordered_id']
            for order_id in order_obj:
                global_order = int(order_id) + iterate_id
                global_order_id_list.append(global_order)
                
            for obj in page['objects']:
                box = page['objects'][obj]
                local_id = int(box['id'])
    
                # 如果box的category_id为2，则保存并迭代
                if box['category_id'] == 2:
                    parent_box = (box['bbox'], local_id + iterate_id)
                # 如果box的category_id为1且box[relations]为空list，则储存的cate为2的为parent
                elif box['category_id'] == 1 and not box['relations']:
                    if parent_box is not None:
                        parent_id = parent_box[1]
                        if parent_id in parent_child_dict:
                            parent_child_dict[parent_id].append(["parent", str(local_id + iterate_id)])
                        else:
                            parent_child_dict[parent_id] = [["parent", str(local_id + iterate_id)]]
                        parent_child_dict[local_id + iterate_id] = [["child", str(parent_id)]]
                id += 1
            iterate_id = id
        
        id = 0
        iterate_id = 0
        i=0
        for page in doc_info[name]['pages']:
            page_info = doc_info[name]['pages'][page]
            for obj in page_info['objects']:
                objt = page_info['objects'][obj]
                global_id_list.append(objt['global_id'])
        
                global_relations = []
                global_gap_dict = {}
                
                relations = objt['relations']
                # gaps = objt['gap']
                
                for relation in relations:
                    # if relation[0] == 'child':
                        global_relation=int(relation[1])+iterate_id
                        global_relations.append(global_relation)
                
                # Adding parent_child_dict info to global_relations
                if id in parent_child_dict:
                    for relation in parent_child_dict[id]:
                        # if relation[0] == 'child':
                            global_relations.append(relation[1])
                global_relations = [int(x) for x in global_relations] 
                relation_list.append(global_relations)

                # 查找当前元素在列表中的位置
                index = global_order_id_list.index(id)
                
                if index == 0:
                    position_list.append([global_order_id_list[index + 1]])
                elif index == len(global_order_id_list) - 1:
                    position_list.append([global_order_id_list[index - 1]])
                else:
                    position_list.append([global_order_id_list[index - 1], global_order_id_list[index + 1]])

                    
                id+=1
            iterate_id=id
            i+=1

    return global_id_list, relation_list, position_list

In [ ]:
global_id_list, relation_list, position_list = combine_infomation(train_doc_info)
train_doc_emb['global_id'] = global_id_list
train_doc_emb['relations'] = relation_list
train_doc_emb['neighbor'] = position_list

global_id_list, relation_list, position_list = combine_infomation(val_doc_info)
val_doc_emb['global_id'] = global_id_list
val_doc_emb['relations'] = relation_list
val_doc_emb['neighbor'] = position_list

In [ ]:
val_doc_emb

In [ ]:
train_doc_emb[40:50]

# Graph making

In [ ]:
!pip install torch_geometric

In [ ]:
import torch
from torch_geometric.data import Data, DataLoader
from torch_geometric.loader import DataLoader
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch_geometric.nn import GATConv
import ast

In [ ]:
def similarity_edges(obj_embeddings, top_k):
    
    norms = obj_embeddings.norm(p=2, dim=1, keepdim=True)
    similarities = torch.matmul(obj_embeddings, obj_embeddings.T) / (norms * norms.T)
    edges = []
    for i in range(len(obj_embeddings)):
        i_similarities = similarities[i]
        top_k_values, top_k_indices = torch.topk(i_similarities, k=top_k)
        
        for j, sim in zip(top_k_indices, top_k_values):
            edges.append((i, j.item(), sim.item()))
    
    return edges

In [ ]:
def relation_edges(obj_embeddings, relations):
    
    edges = []
    for i, relas in enumerate(relations):
        for j in range(obj_embeddings.size(0)):
            if i != j:
                if j in relas:
                    edges.append((i, j, 1))
            else:
                edges.append((i, j, 1))  # (source_node, target_node, similarity, relation)
    return edges

In [ ]:
from tqdm import tqdm 

train_edges_per_doc = {}
train_sim_per_doc = {}
train_neighbor_per_doc = {}

grouped = train_doc_emb.groupby('doc')

for doc_id, group in tqdm(grouped):
    doc_embeddings = torch.cat([torch.tensor(e) for e in group['embeddings']], dim=0)    
    doc_relations = group['relations']
    doc_neighbor = group['neighbor']
    train_edges_per_doc[doc_id] = relation_edges(doc_embeddings, doc_relations)
    train_sim_per_doc[doc_id] = similarity_edges(doc_embeddings, top_k_smilarity)
    train_neighbor_per_doc[doc_id] = relation_edges(doc_embeddings, doc_neighbor)
   
val_edges_per_doc = {}
val_sim_per_doc = {}
val_neighbor_per_doc = {}

# 使用 groupby 根据 doc_id 进行分组
grouped = val_doc_emb.groupby('doc')

for doc_id, group in tqdm(grouped):
    doc_embeddings = torch.cat([torch.tensor(e) for e in group['embeddings']], dim=0)    
    doc_relations = group['relations']
    doc_neighbor = group['neighbor']
    val_edges_per_doc[doc_id] = relation_edges(doc_embeddings, doc_relations)
    val_sim_per_doc[doc_id] = similarity_edges(doc_embeddings, top_k_smilarity)
    val_neighbor_per_doc[doc_id] = relation_edges(doc_embeddings, doc_neighbor)

In [ ]:
for key, value in val_edges_per_doc.items():
    if key == '27760561':
        print(value)
        break

# Data loading

In [ ]:
class PygData(Data):
    def __init__(self, question_embedding, object_embeddings, edge_index1, edge_attr1, edge_index2, edge_attr2, edge_index3, edge_attr3, target):
        super().__init__()
        self.question_embedding = question_embedding
        self.object_embeddings = object_embeddings
        self.edge_index1 = edge_index1
        self.edge_attr1 = edge_attr1
        self.edge_index2 = edge_index2
        self.edge_attr2 = edge_attr2
        self.edge_index3 = edge_index3
        self.edge_attr3 = edge_attr3
        self.target = target

    def __inc__(self, key, value, *args, **kwargs):
        # Increment indices for edge_index1 and edge_index2 (pyg data: only 1 edge default)
        if key in ['edge_index1', 'edge_index2', 'edge_index3']:
            return self.object_embeddings.size(0)  # Increment by the number of nodes
        return super().__inc__(key, value, *args, **kwargs)
        
class QAGNNDataset(torch.utils.data.Dataset):
    def __init__(self, question_embed, object_embeddings, edges_per_doc, sim_per_doc, neighbor_per_doc):
        self.question_embeddings = question_embed.question
        self.answer_ids = question_embed.global_id	
        self.qa_doc_ids = question_embed.pmcid
        # self.obj_doc = object_embeddings.doc
        # self.object_embeddings = object_embeddings.embeddings
        self.obj_doc = object_embeddings
        self.edges_per_doc = edges_per_doc
        self.sim_per_doc = sim_per_doc
        self.neighbor_per_doc = neighbor_per_doc

    def __len__(self):
        return len(self.question_embeddings)

    def __getitem__(self, idx):
        ''' 
        we have to make graph of every question and the objects, so dataloader helped indict the question using idx and we match the doc to find the objects
        '''
        question_embedding = self.question_embeddings[idx].float()
        answer_ids = self.answer_ids[idx]
        answer_doc_id = self.qa_doc_ids[idx]
        
        answer_doc_id = str(answer_doc_id)
        answer_ids = ast.literal_eval(answer_ids)
        
        # qa --> doc_id -->edges
        doc_edges = self.edges_per_doc[answer_doc_id]
        doc_sim = self.sim_per_doc[answer_doc_id]
        doc_neighbor = self.neighbor_per_doc[answer_doc_id]

        # edges --> edges index and attr
        edge_index1 = torch.tensor([[edge[0], edge[1]] for edge in doc_edges], dtype=torch.long).t().contiguous()
        edge_attr1 = torch.tensor([edge[2] for edge in doc_edges], dtype=torch.long)

        edge_index2 = torch.tensor([[edge[0], edge[1]] for edge in doc_sim], dtype=torch.long).t().contiguous()
        edge_attr2 = torch.tensor([edge[2] for edge in doc_sim], dtype=torch.float)

        edge_index3 = torch.tensor([[edge[0], edge[1]] for edge in doc_neighbor], dtype=torch.long).t().contiguous()
        edge_attr3 = torch.tensor([edge[2] for edge in doc_neighbor], dtype=torch.long)
        
        # doc_id --> node feature
        doc_mask = self.obj_doc['doc'] == answer_doc_id
        filtered_embeddings = self.obj_doc.loc[doc_mask, 'embeddings']
        object_embeddings = torch.stack(list(filtered_embeddings)).squeeze(1).float()  #[x, 4096]

        # add QUESTION NODE at the end 
        qa_embeddings = torch.cat([object_embeddings, question_embedding], dim=0) 
        
        # Index of the QUESTION NODE
        question_node_idx = len(filtered_embeddings)  
        num_objects = len(filtered_embeddings)
        question_to_object_edges = torch.tensor([[question_node_idx] * num_objects, torch.arange(num_objects)]).long()
        
        edge_index1 = torch.cat([edge_index1, question_to_object_edges], dim=1)
        edge_index2 = torch.cat([edge_index2, question_to_object_edges], dim=1)
        edge_index3 = torch.cat([edge_index3, question_to_object_edges], dim=1)

        # attr of the QUESTION NODE 
        similarities_question_to_object = torch.matmul(question_embedding, object_embeddings.t()).squeeze(0)
        similarities_question_to_object = torch.tensor(similarities_question_to_object, dtype=torch.float)
        edge_attr_question_to_object = torch.ones(len(filtered_embeddings),  dtype=torch.float)
        
        edge_attr1 = torch.cat([edge_attr1, edge_attr_question_to_object], dim=0)
        edge_attr2 = torch.cat([edge_attr2, similarities_question_to_object], dim=0)        
        edge_attr3 = torch.cat([edge_attr3, edge_attr_question_to_object], dim=0)

        # LABEL
        # doc_id --> node `global_id`
        object_global_id = self.obj_doc.loc[doc_mask, 'global_id']
        object_global_id = torch.tensor(object_global_id.values, dtype=torch.long)
                
        target = torch.zeros(object_global_id.size(0), dtype=torch.float)
        i = 0 
        for id in object_global_id:
            if id in answer_ids:  
                target[i] = 1
            i += 1

        return PygData(question_embedding, qa_embeddings, edge_index1, edge_attr1, edge_index2, edge_attr2, edge_index3, edge_attr3, target)

print('done')

In [ ]:
batch_size = 1
val_dataset = QAGNNDataset(val_question_emb, val_doc_emb, val_edges_per_doc, val_sim_per_doc, val_neighbor_per_doc)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

train_dataset = QAGNNDataset(train_question_emb, train_doc_emb, train_edges_per_doc, train_sim_per_doc, train_neighbor_per_doc)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
len(val_dataloader)

In [ ]:
for idx, batch in enumerate(train_dataloader):
    print(f"Batch {idx + 1}: {batch}")
    break

In [ ]:
for el in val_dataloader:
    print(el['edge_index1'])
    # print(el['edge_attr3'])
    # print(el['question_embedding'].shape)
    # print(el['object_embeddings'].shape)
    # print(el['edge_attr'].shape)
    # print(len(el['answer_ids']))
    break

In [ ]:
# #uncommemt the print line in the QAGNNDataset and print , change the doc name to check the logic
# for index, line in val_doc_emb.iterrows():
#     if line['doc'] == '29609541':
#         fist_emb = line['embeddings'][0]
#         fist_id = line['global_id']
#         print(f'In doc 29609541, the first embeddings is {fist_emb},global_id is {fist_id}')
#         break

# Modeling

In [ ]:
import torch
from torch import nn
from torch_geometric.nn import MessagePassing
from torch_geometric.nn import knn_graph
from torch_geometric.utils import softmax


class GATConvC(MessagePassing):
    def __init__(self, in_channels, out_channels, head_count=4, aggr='add'):
        super().__init__(aggr=aggr)  
        self.sim_weights = nn.Parameter(torch.full((1,), 0.5))
        self.head_count = head_count
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.dim_per_head = out_channels // head_count

        assert out_channels % head_count == 0, "out_channels must be divisible by head_count"

        self.linear_key = nn.Linear(in_channels + 1, head_count * self.dim_per_head)
        self.linear_query = nn.Linear(in_channels, head_count * self.dim_per_head)
        self.linear_msg = nn.Linear(in_channels + 1, head_count * self.dim_per_head)

        self.mlp = nn.Sequential(
            nn.Linear(out_channels, out_channels),
            nn.ReLU(),
            nn.Linear(out_channels, out_channels)
        )
    
    def forward(self, x, edge_index, edge_attr):

        aggr_out = self.propagate(edge_index, x=x, edge_attr=edge_attr) #[E, emb_dim]
        out = self.mlp(aggr_out)
        
        return out

    def message(self, x_i, x_j, edge_attr, edge_index):

        # edge_attr = self.sim_weights * edge_attr[:,0] + (1 - self.sim_weights) * edge_attr[:,1]
        # edge_attr = edge_attr[:,1]
        edge_attr = edge_attr.view(-1, 1)

        edge_features = torch.cat([x_i, x_j, edge_attr], dim=1)  

        key_input = torch.cat([x_i, edge_attr], dim=1)  
        msg_input = torch.cat([x_j, edge_attr], dim=1)  
        # Linear transformations
        key = self.linear_key(key_input).view(-1, self.head_count, self.dim_per_head)  
        msg = self.linear_msg(msg_input).view(-1, self.head_count, self.dim_per_head) 
        query = self.linear_query(x_j).view(-1, self.head_count, self.dim_per_head) 
        
        # attn score
        scores = (query * key).sum(dim=-1)
        scores = scores / (self.dim_per_head ** 0.5)  
        # scores = F.leaky_relu(scores, negative_slope=0.2)
        alpha = softmax(scores, edge_index[0])
        #added attr inside
        # alpha = edge_attr * alpha
        # alpha = torch.sigmoid(alpha)
        self._alpha = alpha
        
        out = msg * alpha.unsqueeze(-1)
        return out.view(-1, self.out_channels)


In [ ]:
import math

def gelu(x):
    """ Implementation of the gelu activation function currently in Google Bert repo (identical to OpenAI GPT).
        Also see https://arxiv.org/abs/1606.08415
    """
    return 0.5 * x * (1 + torch.tanh(math.sqrt(2 / math.pi) * (x + 0.044715 * torch.pow(x, 3))))


class GELU(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return gelu(x)

class QAGNN_Message_Passing(nn.Module):
    def __init__(self, input_dim, hidden_dim, activation=GELU(), dropout=0.1):
        super(QAGNN_Message_Passing, self).__init__()
        # Using GATConv (could replace with GATv2Conv if needed)
        self.gat = GATConvC(input_dim, hidden_dim)
        self.bn = nn.BatchNorm1d(hidden_dim)
        self.act = activation

    def forward(self, x, edge_index, edge_attr):
        x = self.gat(x, edge_index, edge_attr)
        x = self.bn(x)
        x = self.act(x)
        return x

class QAGNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, k, dropout=0.1):
        super().__init__()
        self.vecmap = nn.Linear(input_size, hidden_size)

        self.gnn = QAGNN_Message_Passing(hidden_size, hidden_size, activation=GELU())
        self.linear = nn.Linear(output_size, 1)
        self.weight_sim = nn.Parameter(torch.tensor(0.5))
        self.weight_rela = nn.Parameter(torch.tensor(0.5))
    
    def forward(self, node_features, edge_index1, edge_attr1, edge_index2, edge_attr2):
        node_features = self.vecmap(node_features)

        node_sim_conv = self.gnn(node_features, edge_index2, edge_attr2)
        node_rela_conv = self.gnn(node_features, edge_index1, edge_attr1)

        node_score_sim = self.linear(node_sim_conv)
        node_score_rela = self.linear(node_rela_conv)
        node_score = self.weight_sim * node_score_sim + self.weight_rela * node_score_rela
        
        return node_score[:-1]

In [ ]:
def calculate_exact_match_ratio(true_labels, predicted_labels):
    exact_match = 0
    for i in range(len(true_labels)):
      if true_labels[i] == predicted_labels[i]:
        exact_match += 1
    return exact_match / len(true_labels)

def calculate_target_match_ratio(true_labels, predicted_labels):
    exact_match = 0
    for i in range(len(true_labels)):
      if true_labels[i] == predicted_labels[i] == 1.0:
        exact_match += 1
                  
    return exact_match / np.sum(true_labels.tolist())

In [ ]:
def validate(model, val_dataloader):
    model.eval()
    exact_match_sum = 0
    target_match_sum = []
    total_batches = 0
    running_loss = 0.0
    
    predict_list = []
    target_list = []
    with torch.no_grad():
        for batch in tqdm(val_dataloader):
            object_embeddings = batch.object_embeddings.to(device)
            edge_index1 = batch.edge_index1.to(device) #relation 
            edge_attr1 = batch.edge_attr1.to(device)
            edge_index2 = batch.edge_index2.to(device) #similarity
            edge_attr2 = batch.edge_attr2.to(device)
            edge_index3 = batch.edge_index3.to(device) #neighbor
            edge_attr3 = batch.edge_attr3.to(device)
            question_embedding = batch.question_embedding.to(device)
            target = batch.target.to(device)
    
            # optimizer.zero_grad()
            logits = model(object_embeddings, edge_index1, edge_attr1, edge_index2, edge_attr2)
            probabilities = torch.sigmoid(logits)
            preds = (probabilities >= 0.5).float()
            
            loss = loss_fn(logits.squeeze(), target)

            running_loss += loss.item()

            # Exact Match of every ele
            exact_match = calculate_exact_match_ratio(target, preds)
            exact_match_sum += exact_match

            # Positive Recall
            if torch.sum(target).item() > 0: 
                target_match = calculate_target_match_ratio(target, preds)
                target_match_sum.append(target_match)

            total_batches += 1

            predict_list.append(preds.squeeze().to("cpu").tolist())
            target_list.append(target.to("cpu").to(torch.int).tolist())

    avg_loss = running_loss / total_batches
    avg_val_acc = exact_match_sum / total_batches
    avg_pos_recall = sum(target_match_sum) / len(target_match_sum) if target_match_sum else 0.0
    epoch_accu = calculate_exact_match_ratio(target_list, predict_list)

    print(f"Epoch [{epoch+1}/{num_epochs}],Val Avg Loss: {avg_loss:.4f}, Val Accuracy: {avg_val_acc:.4f}, Val Recall: {avg_pos_recall:.4f}, Val Exact match:{epoch_accu:.4f}")
    return epoch_accu

# Training

In [ ]:
class FocalLoss(nn.Module):
    def __init__(self, gamma=2):
        super(FocalLoss, self).__init__()
        self.gamma = gamma

    def forward(self, logits, targets):
        probabilities = torch.sigmoid(logits)
        
        bce_loss = F.binary_cross_entropy_with_logits(logits, targets, reduction='none')
        pt = probabilities * targets + (1 - probabilities) * (1 - targets)
        focal_loss = (1 - pt) ** self.gamma * bce_loss
        
        return focal_loss.mean()

In [ ]:
class weighted_bce_loss(nn.Module):
    def __init__(self, weight=2.0):
        self.weight = weight
            
    def forward(self, logits, targets):
        pos_weight = torch.tensor([self.weight], dtype=torch.float32, device=logits.device)
        loss = F.binary_cross_entropy_with_logits(logits, targets, pos_weight=pos_weight)
        
        return loss

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

input_size = 4096
hidden_size = 4096
k = 1  # Number of GAT layers

# pos_weight = torch.tensor([2.0]).to(device)
model = QAGNN(input_size, hidden_size,  hidden_size, k).to(device)
for param in model.parameters():
    param.requires_grad = True

if loss_set == 'bce':
    loss_fn=nn.BCEWithLogitsLoss()
if loss_set == 'focal':
    loss_fn = FocalLoss(gamma=2)
if loss_set == 'weighted_bce':
    loss_fn = weighted_bce_loss(weight=2.0)
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-05)

In [ ]:
model

In [ ]:
# start_model_path = '/kaggle/input/3graph-mlp/best_model (4).pth'
# model.load_state_dict(torch.load(start_model_path))

In [ ]:
import torch
import torch.nn.functional as F
from tqdm import tqdm

num_epochs = 15
all_pred = {}
best_val_acc = 0.0  
best_model_path = "/kaggle/working/best_model.pth" 

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    exact_match_sum = 0
    total_batches = 0
    target_match_sum = []

    #Exact match
    predict_list = []
    target_list = []
    
    for i, batch in enumerate(tqdm(train_dataloader)):
        optimizer.zero_grad()
        object_embeddings = batch.object_embeddings.to(device)
        edge_index1 = batch.edge_index1.to(device) #relation 
        edge_attr1 = batch.edge_attr1.to(device)
        edge_index2 = batch.edge_index2.to(device) #similarity
        edge_attr2 = batch.edge_attr2.to(device)
        edge_index3 = batch.edge_index3.to(device) #neighbor
        edge_attr3 = batch.edge_attr3.to(device)
        question_embedding = batch.question_embedding.to(device)
        target = batch.target.to(device)

        logits = model(object_embeddings, edge_index1, edge_attr1, edge_index2, edge_attr2)
        loss = loss_fn(logits.squeeze(), target)

        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

        # Performance of every element(every obj)
        probabilities = torch.sigmoid(logits)
        preds = (probabilities > 0.5).float()
        exact_match = calculate_exact_match_ratio(target, preds)
        exact_match_sum += exact_match
        # print(logits)
        # print(loss)
        # print(probabilities)

        # if question has the true answer comupte the RECALL
        if torch.sum(target).item()>0:
            target_match = calculate_target_match_ratio(target, preds)
            target_match_sum.append(target_match)
            # print(f'target match:{target_match}, exact_match:{exact_match}')

        # Performance of Exactly match of qa(obj list)
        predict_list.append(preds.squeeze().to("cpu").tolist())
        target_list.append(target.to("cpu").to(torch.int).tolist())

        total_batches += 1
                        
    #performance of obj
    avg_loss = running_loss / total_batches
    avg_acc = exact_match_sum / total_batches
    avg_acc_pos = sum(target_match_sum) / len(target_match_sum) if target_match_sum else 0.0
    #performance of exact match of obj list
    epoch_accu = calculate_exact_match_ratio(target_list, predict_list)

    print(f"Epoch [{epoch+1}/{num_epochs}],Train Avg Loss: {avg_loss:.4f}, Train Accuracy: {avg_acc:.4f}, Train Recall: {avg_acc_pos:.4f}, Exact match:{epoch_accu:.4f}")

    val_acc = validate(model, val_dataloader)

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), best_model_path)
        print(f"New best model saved with avg of Exact match: {best_val_acc:.4f}")
    

# test

In [ ]:
class PygtestData(Data):
    def __init__(self, question_embedding, object_embeddings, edge_index1, edge_attr1, edge_index2, edge_attr2, edge_index3, edge_attr3, global_id):
        super().__init__()
        self.question_embedding = question_embedding
        self.object_embeddings = object_embeddings
        self.edge_index1 = edge_index1
        self.edge_attr1 = edge_attr1
        self.edge_index2 = edge_index2
        self.edge_attr2 = edge_attr2
        self.edge_index3 = edge_index3
        self.edge_attr3 = edge_attr3
        self.global_id = global_id

    def __inc__(self, key, value, *args, **kwargs):
        # Increment indices for edge_index1 and edge_index2 (pyg data: only 1 edge default)
        if key in ['edge_index1', 'edge_index2', 'edge_index3']:
            return self.object_embeddings.size(0)  # Increment by the number of nodes
        return super().__inc__(key, value, *args, **kwargs)
        
class testDataset(torch.utils.data.Dataset):
    def __init__(self, question_embed, object_embeddings, edges_per_doc, sim_per_doc, neighbor_per_doc):
        self.question_embeddings = question_embed.question
        self.qa_doc_ids = question_embed.pmcid
        # self.obj_doc = object_embeddings.doc
        # self.object_embeddings = object_embeddings.embeddings
        self.obj_doc = object_embeddings
        self.edges_per_doc = edges_per_doc
        self.sim_per_doc = sim_per_doc
        self.neighbor_per_doc = neighbor_per_doc

    def __len__(self):
        return len(self.question_embeddings)

    def __getitem__(self, idx):
        ''' 
        we have to make graph of every question and the objects, so dataloader helped indict the question using idx and we match the doc to find the objects
        '''
        question_embedding = self.question_embeddings[idx].float()
        answer_doc_id = self.qa_doc_ids[idx]
        
        answer_doc_id = str(answer_doc_id)
        
        # qa --> doc_id -->edges
        doc_edges = self.edges_per_doc[answer_doc_id]
        doc_sim = self.sim_per_doc[answer_doc_id]
        doc_neighbor = self.neighbor_per_doc[answer_doc_id]

        # edges --> edges index and attr
        edge_index1 = torch.tensor([[edge[0], edge[1]] for edge in doc_edges], dtype=torch.long).t().contiguous()
        edge_attr1 = torch.tensor([edge[2] for edge in doc_edges], dtype=torch.long)

        edge_index2 = torch.tensor([[edge[0], edge[1]] for edge in doc_sim], dtype=torch.long).t().contiguous()
        edge_attr2 = torch.tensor([edge[2] for edge in doc_sim], dtype=torch.float)

        edge_index3 = torch.tensor([[edge[0], edge[1]] for edge in doc_neighbor], dtype=torch.long).t().contiguous()
        edge_attr3 = torch.tensor([edge[2] for edge in doc_neighbor], dtype=torch.long)
        
        # doc_id --> node feature
        doc_mask = self.obj_doc['doc'] == answer_doc_id
        filtered_embeddings = self.obj_doc.loc[doc_mask, 'embeddings']
        object_embeddings = torch.stack(list(filtered_embeddings)).squeeze(1).float()  #[x, 4096]

        # add QUESTION NODE at the end 
        qa_embeddings = torch.cat([object_embeddings, question_embedding], dim=0) 
        
        # Index of the QUESTION NODE
        question_node_idx = len(filtered_embeddings)  
        num_objects = len(filtered_embeddings)
        question_to_object_edges = torch.tensor([[question_node_idx] * num_objects, torch.arange(num_objects)]).long()
        
        edge_index1 = torch.cat([edge_index1, question_to_object_edges], dim=1)
        edge_index2 = torch.cat([edge_index2, question_to_object_edges], dim=1)
        edge_index3 = torch.cat([edge_index3, question_to_object_edges], dim=1)

        # attr of the QUESTION NODE 
        similarities_question_to_object = torch.matmul(question_embedding, object_embeddings.t()).squeeze(0)  # 点乘计算相似度
        similarities_question_to_object = torch.tensor(similarities_question_to_object, dtype=torch.float)
        edge_attr_question_to_object = torch.ones(len(filtered_embeddings),  dtype=torch.float)
        
        edge_attr1 = torch.cat([edge_attr1, edge_attr_question_to_object], dim=0)
        edge_attr2 = torch.cat([edge_attr2, similarities_question_to_object], dim=0)        
        edge_attr3 = torch.cat([edge_attr3, edge_attr_question_to_object], dim=0)

        #global_id
        global_id = self.obj_doc.loc[doc_mask, 'global_id'].values
        global_id = torch.tensor(global_id)

        return PygtestData(question_embedding, qa_embeddings, edge_index1, edge_attr1, edge_index2, edge_attr2, edge_index3, edge_attr3, global_id)

print('done')

In [ ]:
def test(model, val_dataloader):
    model.eval()
    exact_match_sum = 0
    target_match_sum = []
    total_batches = 0
    running_loss = 0.0
    
    predict_list = []
    global_id_list = []
    with torch.no_grad():
        for batch in tqdm(val_dataloader):
            object_embeddings = batch.object_embeddings.to(device)
            edge_index1 = batch.edge_index1.to(device) #relation 
            edge_attr1 = batch.edge_attr1.to(device)
            edge_index2 = batch.edge_index2.to(device) #similarity
            edge_attr2 = batch.edge_attr2.to(device)
            edge_index3 = batch.edge_index3.to(device) #neighbor
            edge_attr3 = batch.edge_attr3.to(device)
            question_embedding = batch.question_embedding.to(device)
            global_id = batch.global_id.to(device)
    
            # optimizer.zero_grad()
            logits = model(object_embeddings, edge_index1, edge_attr1, edge_index2, edge_attr2)
            probabilities = torch.sigmoid(logits)
            preds = (probabilities > 0.5).float()
            
            predict_list.append(preds.squeeze().to("cpu").tolist())
            global_id_list.append(global_id.to("cpu").to(torch.int).tolist())

    return predict_list, global_id_list

In [ ]:
def extracting(pre_ids, global_ids):
  outputs = []
  for j, pids in enumerate(pre_ids):
    output = []
    gids = global_ids[j]
    for i,pid in enumerate(pids):
      if pid == 1:
        output.append(gids[i])
    outputs.append(output)
  return outputs

In [ ]:
# best_model_path = '/kaggle/input/rela-sim-graph/relasim-graph.pth'
model.load_state_dict(torch.load(best_model_path))

In [ ]:
global_id_list, relation_list, position_list = combine_infomation(test_doc_info)
test_doc_emb['global_id'] = global_id_list
test_doc_emb['relations'] = relation_list
test_doc_emb['neighbor'] = position_list

grouped = test_doc_emb.groupby('doc')

test_edges_per_doc = {}
test_sim_per_doc = {}
test_neighbor_per_doc = {}

for doc_id, group in tqdm(grouped):
    doc_embeddings = torch.cat([torch.tensor(e) for e in group['embeddings']], dim=0)    
    doc_relations = group['relations']
    doc_neighbor = group['neighbor']
    test_edges_per_doc[doc_id] = relation_edges(doc_embeddings, doc_relations)
    test_sim_per_doc[doc_id] = similarity_edges(doc_embeddings, top_k_smilarity)
    test_neighbor_per_doc[doc_id] = relation_edges(doc_embeddings, doc_neighbor)

test_dataset = testDataset(test_question_emb, test_doc_emb, test_edges_per_doc, test_sim_per_doc, test_neighbor_per_doc)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
predict_list, global_id_list = test(model, test_dataloader)
output = extracting(predict_list, global_id_list)

new_output = []
for i in range(len(output)):
    if output[i] == []:
        new_output.append([-1])
    elif -1 in output[i] and len(output[i]) > 1:
        correct_elements = []
        for e in output[i]:
            if e != -1:
                correct_elements.append(e)
        new_output.append(str(correct_elements))
    else:
        new_output.append(str(output[i]))

output = new_output

id_list = range(0, len(output))
df = pd.DataFrame(id_list, columns=['id'])
df['answer'] = output
df.to_csv(f'/kaggle/working/submission.csv', index=False)